In [3]:
%pip install git+https://github.com/espnet/espnet
%pip install -q espnet_model_zoo

  Cloning https://github.com/espnet/espnet to /tmp/pip-req-build-b9aq3yie
  Running command git clone --filter=blob:none --quiet https://github.com/espnet/espnet /tmp/pip-req-build-b9aq3yie
  Resolved https://github.com/espnet/espnet to commit ebdd5c792a5003dacc7f149f043835727f927be4
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 14.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 23.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.0/212.0 kB 37.5 MB/s eta 0:00:00
  Installing build

In [7]:
import os

import soundfile
from IPython.display import display, Audio

# this is already a single channel audio file
WAV_FILE = '2_noipd.wav'
mixwav_mc, sr = soundfile.read(WAV_FILE)
# mixwav_sc = mixwav_mc[:,1]

filename_prefix = os.path.splitext(os.path.basename(WAV_FILE))[0]

In [8]:
#@title Choose Speech Enhancement model { run: "auto" }

fs = 16000 #@param {type:"integer"}
tag = "espnet/Wangyou_Zhang_chime4_enh_train_enh_beamformer_mvdr_raw" #@param ["espnet/Wangyou_Zhang_chime4_enh_train_enh_beamformer_mvdr_raw", "espnet/Wangyou_Zhang_chime4_enh_train_enh_dc_crn_mapping_snr_raw", "lichenda/chime4_fasnet_dprnn_tac", "https://zenodo.org/record/6025881/files/enh_train_enh_beamformer_mvdr_raw_valid.si_snr.ave.zip"]

In [9]:
from espnet_model_zoo.downloader import ModelDownloader

d = ModelDownloader()
cfg = d.download_and_unpack("espnet/Wangyou_Zhang_chime4_enh_train_enh_conv_tasnet_raw")

In [11]:
# For models uploaded to Zenodo or HuggingFace, you can use the following python script instead:
import sys
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.enh_inference import SeparateSpeech

d = ModelDownloader()

cfg = d.download_and_unpack(tag)
enh_model_mc = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=4,
  normalize_output_wav=True,
  device="cuda:0",
)

In [12]:
wave = enh_model_mc(mixwav_mc[None, ...], sr)
print("Input real noisy speech", flush=True)
display(Audio(mixwav_mc.T, rate=sr))

print(f"Enhanced speech with model {tag}", flush=True)
display(Audio(wave[0].squeeze(), rate=sr))          

# export the enhanced speech
dir = 'audios'
output_path = os.path.join(dir, f"{filename_prefix}_enhanced.wav")
soundfile.write(output_path, wave[0].squeeze(), sr)

/home/kangying/anaconda3/envs/pyannote/lib/python3.8/site-packages/espnet2/layers/stft.py:164: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.n_fft) // self.hop_length + 1


IndexError: index 3 is out of bounds for dimension 1 with size 2